In [1]:
setwd('..')

In [3]:
library(dplyr)


Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union




In [4]:
years = 2016:2023
conf_name = '20240216-5'

## Read data

In [19]:
results_orig = read.csv(sprintf('results/%s/ci_summary_avg.csv', conf_name))
dim(results_orig)
head(results_orig)

[1] 18431    20

,Actual,Pred,Pred.lower,Pred.upper,Pred.sd,AbsEffect,AbsEffect.lower,AbsEffect.upper,AbsEffect.sd,RelEffect,RelEffect.lower,RelEffect.upper,RelEffect.sd,alpha,p,obs_id,event_date,n_events,max_score,sum_score
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<int>,<dbl>,<dbl>
1,0.8687154,0.9986452,0.3831622,1.632240,0.3158630,-0.12992989,-0.7635249,0.4855532,0.3158630,-0.04878256,-0.4699611,1.2395421,0.8940162,0.05,0.3138462,AALI,2016-01-05,1,0.7055333,0.7055333
2,0.9125030,1.0295840,0.3104252,1.714852,0.3778008,-0.11708100,-0.8023490,0.6020778,0.3778008,-0.08958057,-0.4705635,1.8153162,6.2009318,0.05,0.3956159,AALI,2016-03-14,1,0.7534191,0.7534191
3,0.9591118,0.9945437,0.2731302,1.741315,0.3696962,-0.03543194,-0.7822036,0.6859816,0.3696962,0.28002031,-0.4528748,2.3188671,2.2395958,0.05,0.4754300,AALI,2016-04-13,1,0.7116593,0.7116593
4,0.9591118,0.9945437,0.3284059,1.714351,0.3474891,-0.03543194,-0.7552394,0.6307058,0.3474891,0.13845476,-0.4405395,1.9210084,0.7573462,0.05,0.4312039,AALI,2016-04-14,1,0.6993880,0.6993880
5,0.7238977,1.0005072,0.2951317,1.623820,0.3420472,-0.27660957,-0.8999218,0.4287659,0.3420472,0.14348789,-0.5575325,1.2943987,9.3909166,0.05,0.2214212,AALI,2016-04-25,1,0.8159519,0.8159519
6,0.6686964,0.9907027,0.3192312,1.656398,0.3498839,-0.32200627,-0.9877013,0.3494652,0.3498839,-0.28581358,-0.6133736,0.8777734,2.7191069,0.05,0.1893861,AALI,2016-05-03,1,0.7363428,0.7363428


In [20]:
table(results_orig$n_events)


    1     2     3     4     6 
16614  1567   218    31     1 

In [21]:
table(lubridate::year(results_orig$event_date))


2016 2017 2018 2019 2020 2021 2022 2023 
1218 2250 2776 1541 2476 2496 3523 2151 

In [22]:
length(unique(results_orig$obs_id))

[1] 768

In [23]:
head(sort(table(results_orig$obs_id), decreasing=TRUE), 30)


ADRO PTBA BBRI TLKM WSKT BBNI BMRI WIKA BBCA ASII PTPP PGAS UNVR ANTM ICBP ITMG 
 389  373  332  330  311  302  291  288  282  277  272  264  261  252  247  241 
UNTR JSMR ADHI KLBF HMSP INDY INDF SMGR AKRA GGRM INTP MEDC AALI CTRA 
 232  226  213  209  208  201  198  196  187  186  184  183  177  173 

## Process CausalImpact results

In [24]:
results = results_orig
results_filt = results %>% filter(max_score >= quantile(max_score, 0.5))
results_filt1 = results %>% filter(max_score >= quantile(max_score, 0.1))
results_grouped = results %>%
    mutate(event_year=lubridate::year(event_date), Effect=Actual - Pred) %>%
    group_by(obs_id, event_year) %>%
    summarise(n=n(), mean_p=mean(p), sd_p=sd(p), mean_eff=mean(Effect), sd_eff=sd(Effect)) %>%
    group_by(obs_id) %>% filter(n() > 1) %>%    # discard firms which only appeared in 1 year
    arrange(obs_id, event_year) %>%
    plm::pdata.frame(index=c('obs_id', 'event_year'))
dim(results)
head(results)

`summarise()` has grouped output by 'obs_id'. You can override using the `.groups` argument.


[1] 18431    20

,Actual,Pred,Pred.lower,Pred.upper,Pred.sd,AbsEffect,AbsEffect.lower,AbsEffect.upper,AbsEffect.sd,RelEffect,RelEffect.lower,RelEffect.upper,RelEffect.sd,alpha,p,obs_id,event_date,n_events,max_score,sum_score
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<int>,<dbl>,<dbl>
1,0.8687154,0.9986452,0.3831622,1.632240,0.3158630,-0.12992989,-0.7635249,0.4855532,0.3158630,-0.04878256,-0.4699611,1.2395421,0.8940162,0.05,0.3138462,AALI,2016-01-05,1,0.7055333,0.7055333
2,0.9125030,1.0295840,0.3104252,1.714852,0.3778008,-0.11708100,-0.8023490,0.6020778,0.3778008,-0.08958057,-0.4705635,1.8153162,6.2009318,0.05,0.3956159,AALI,2016-03-14,1,0.7534191,0.7534191
3,0.9591118,0.9945437,0.2731302,1.741315,0.3696962,-0.03543194,-0.7822036,0.6859816,0.3696962,0.28002031,-0.4528748,2.3188671,2.2395958,0.05,0.4754300,AALI,2016-04-13,1,0.7116593,0.7116593
4,0.9591118,0.9945437,0.3284059,1.714351,0.3474891,-0.03543194,-0.7552394,0.6307058,0.3474891,0.13845476,-0.4405395,1.9210084,0.7573462,0.05,0.4312039,AALI,2016-04-14,1,0.6993880,0.6993880
5,0.7238977,1.0005072,0.2951317,1.623820,0.3420472,-0.27660957,-0.8999218,0.4287659,0.3420472,0.14348789,-0.5575325,1.2943987,9.3909166,0.05,0.2214212,AALI,2016-04-25,1,0.8159519,0.8159519
6,0.6686964,0.9907027,0.3192312,1.656398,0.3498839,-0.32200627,-0.9877013,0.3494652,0.3498839,-0.28581358,-0.6133736,0.8777734,2.7191069,0.05,0.1893861,AALI,2016-05-03,1,0.7363428,0.7363428


In [25]:
table(results$event_year)

< table of extent 0 >

In [26]:
length(unique(results$obs_id))

[1] 768

In [27]:
table(table(results$obs_id))


  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18  19  20 
110  81  82  46  57  40  29  24  31  17  15  14   9   9   2  14   6   8   6   2 
 21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36  37  38  39  40 
  5   4   7   4   1   1   3   3   3   7   2   2   7   2   5   4   2   5   1   1 
 41  42  43  44  45  46  48  50  52  53  54  57  58  60  61  63  64  65  66  69 
  1   1   1   2   1   2   1   1   2   3   1   1   1   2   2   3   1   1   1   1 
 70  77  79  80  81  84  87  89  91  93  97  99 100 101 104 105 107 110 111 112 
  2   4   1   1   2   1   1   1   1   1   2   1   1   1   1   1   1   2   1   1 
119 126 130 135 143 149 164 168 173 177 183 184 186 187 196 198 201 208 209 213 
  1   1   1   3   1   2   1   1   1   1   1   1   1   1   1   1   1   1   1   1 
226 232 241 247 252 261 264 272 277 282 288 291 302 311 330 332 373 389 
  1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1 

In [28]:
plm::punbalancedness(results_grouped)

gamma        nu 
0.7865587 0.8168654

## Test hypothesis

### default

In [42]:
mean((results$p < 0.05) & (results$Actual > results$Pred))
mean((results_filt$p < 0.05) & (results_filt$Actual > results_filt$Pred))
mean((results_filt1$p < 0.05) & (results_filt1$Actual > results_filt1$Pred))

[1] 0.1795345

[1] 0.1821832

[1] 0.1777188

In [43]:
mean((results$p < 0.05) & (results$Actual < results$Pred))
mean((results_filt$p < 0.05) & (results_filt$Actual < results_filt$Pred))
mean((results_filt1$p < 0.05) & (results_filt1$Actual < results_filt1$Pred))

[1] 0.09749878

[1] 0.09646267

[1] 0.09862551

In [44]:
mean(results$p >= 0.05)
mean(results_filt$p >= 0.05)
mean(results_filt1$p >= 0.05)

[1] 0.7229667

[1] 0.7213542

[1] 0.7236557

### dummy

In [26]:
mean((results$p < 0.05) & (results$Actual > results$Pred))
mean((results_filt$p < 0.05) & (results_filt$Actual > results_filt$Pred))
mean((results_filt1$p < 0.05) & (results_filt1$Actual > results_filt1$Pred))

[1] 0.1616796

[1] 0.1568199

[1] 0.1594467

In [27]:
mean((results$p < 0.05) & (results$Actual < results$Pred))
mean((results_filt$p < 0.05) & (results_filt$Actual < results_filt$Pred))
mean((results_filt1$p < 0.05) & (results_filt1$Actual < results_filt1$Pred))

[1] 0.1045648

[1] 0.1029813

[1] 0.1043563

In [29]:
mean(results$p >= 0.05)
mean(results_filt$p >= 0.05)
mean(results_filt1$p >= 0.05)

[1] 0.7337556

[1] 0.7401988

[1] 0.7361971

### dummy2

In [ ]:
mean(results$Actual > results$Pred)
mean(results_filt$Actual > results_filt$Pred)
mean(results_filt1$Actual > results_filt1$Pred)

[1] 0.3933166

[1] 0.398452

[1] 0.391594

In [ ]:
mean((results$p < 0.05) & (results$Actual > results$Pred))
mean((results_filt$p < 0.05) & (results_filt$Actual > results_filt$Pred))
mean((results_filt1$p < 0.05) & (results_filt1$Actual > results_filt1$Pred))

[1] 0.1577628

[1] 0.1592718

[1] 0.1566134

In [ ]:
mean((results$p < 0.05) & (results$Actual < results$Pred))
mean((results_filt$p < 0.05) & (results_filt$Actual < results_filt$Pred))
mean((results_filt1$p < 0.05) & (results_filt1$Actual < results_filt1$Pred))

[1] 0.1083733

[1] 0.1022566

[1] 0.1075581

In [ ]:
mean(results$p >= 0.05)
mean(results_filt$p >= 0.05)
mean(results_filt1$p >= 0.05)

[1] 0.7338639

[1] 0.7384716

[1] 0.7358285

### dummy3

In [16]:
mean(results$Actual > results$Pred)
mean(results_filt$Actual > results_filt$Pred)
mean(results_filt1$Actual > results_filt1$Pred)

[1] 0.3999673

[1] 0.4011133

[1] 0.3996725

In [17]:
mean((results$p < 0.05) & (results$Actual > results$Pred))
mean((results_filt$p < 0.05) & (results_filt$Actual > results_filt$Pred))
mean((results_filt1$p < 0.05) & (results_filt1$Actual > results_filt1$Pred))

[1] 0.163174

[1] 0.1649203

[1] 0.1634831

In [18]:
mean((results$p < 0.05) & (results$Actual < results$Pred))
mean((results_filt$p < 0.05) & (results_filt$Actual < results_filt$Pred))
mean((results_filt1$p < 0.05) & (results_filt1$Actual < results_filt1$Pred))

[1] 0.103962

[1] 0.1016154

[1] 0.1036929

In [19]:
mean(results$p >= 0.05)
mean(results_filt$p >= 0.05)
mean(results_filt1$p >= 0.05)

[1] 0.732864

[1] 0.7334643

[1] 0.732824

### dummy4

In [15]:
mean(results$Actual > results$Pred)
mean(results_filt$Actual > results_filt$Pred)
mean(results_filt1$Actual > results_filt1$Pred)

[1] 0.3903834

[1] 0.3868438

[1] 0.3875476

In [16]:
mean((results$p < 0.05) & (results$Actual > results$Pred))
mean((results_filt$p < 0.05) & (results_filt$Actual > results_filt$Pred))
mean((results_filt1$p < 0.05) & (results_filt1$Actual > results_filt1$Pred))

[1] 0.1558484

[1] 0.1568286

[1] 0.1553821

In [17]:
mean((results$p < 0.05) & (results$Actual < results$Pred))
mean((results_filt$p < 0.05) & (results_filt$Actual < results_filt$Pred))
mean((results_filt1$p < 0.05) & (results_filt1$Actual < results_filt1$Pred))

[1] 0.107384

[1] 0.1081464

[1] 0.1084891

In [18]:
mean(results$p >= 0.05)
mean(results_filt$p >= 0.05)
mean(results_filt1$p >= 0.05)

[1] 0.7367676

[1] 0.735025

[1] 0.7361288

### dummy5

In [17]:
mean(results$Actual > results$Pred)
mean(results_filt$Actual > results_filt$Pred)
mean(results_filt1$Actual > results_filt1$Pred)

[1] 0.3956391

[1] 0.3944844

[1] 0.3943671

In [18]:
mean((results$p < 0.05) & (results$Actual > results$Pred))
mean((results_filt$p < 0.05) & (results_filt$Actual > results_filt$Pred))
mean((results_filt1$p < 0.05) & (results_filt1$Actual > results_filt1$Pred))

[1] 0.1638594

[1] 0.1602354

[1] 0.1621442

In [19]:
mean((results$p < 0.05) & (results$Actual < results$Pred))
mean((results_filt$p < 0.05) & (results_filt$Actual < results_filt$Pred))
mean((results_filt1$p < 0.05) & (results_filt1$Actual < results_filt1$Pred))

[1] 0.1047697

[1] 0.1052976

[1] 0.1041793

In [20]:
mean(results$p >= 0.05)
mean(results_filt$p >= 0.05)
mean(results_filt1$p >= 0.05)

[1] 0.7313709

[1] 0.734467

[1] 0.7336766

## t-test

In [ ]:
results = read.csv('results/20240216-5/ci_summary_avg.csv')
results_dummy = bind_rows(lapply(1:5, function(x) read.csv(sprintf('results/20240216-5-dummy%s/ci_summary_avg.csv', x))))

dim(results)
dim(results_dummy)

In [27]:
z = (results$p < 0.05) & (results$Actual > results$Pred)
mean(z)
sd(z)
nrow(results)

[1] 0.1795345

[1] 0.3838096

[1] 18431

In [28]:
z_dummy = (results_dummy$p < 0.05) & (results_dummy$Actual > results_dummy$Pred)
mean(z_dummy)
sd(z_dummy)
nrow(results_dummy)

[1] 0.1589539

[1] 0.3656351

[1] 91731

### vs dummies

In [43]:
t.test(z, z_dummy, alternative = "two.sided", var.equal = FALSE)


	Welch Two Sample t-test

data:  z and z_dummy
t = 6.6949, df = 25593, p-value = 2.203e-11
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
 0.01455523 0.02660593
sample estimates:
mean of x mean of y 
0.1795345 0.1589539 


In [41]:
(mean(z) - mean(z_dummy)) / sqrt(var(z)/length(z) + var(z_dummy)/length(z_dummy))

[1] 6.694897